### Take keras models and get tensorflow checkpoints
This is a variation of: https://github.com/amir-abdi/keras_to_tensorflow/blob/master/keras_to_tensorflow.ipynb

To port the weights from the TensorFlow checkpoint to deeplearn.js run:

`python dump_checkpoint_vars.py --output_dir=models/hamlet/ --checkpoint_file=keras_to_tf/tf_models/hamlet.ckpt`

In [16]:
from keras.models import load_model
import tensorflow as tf
import os
import os.path as osp
from keras import backend as K

In [18]:
# Set parameters
model_name = 'hamlet'

input_fld = 'keras_models'
weight_file = model_name + '.h5'
num_output = 1
write_graph_def_ascii_flag = True
prefix_output_node_names_of_final_network = 'output_node'
output_graph_name = model_name + '.pb'

output_fld = 'tf_models/'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
weight_file_path = osp.join(input_fld, weight_file)

In [21]:
# Load keras model and rename output
K.set_learning_phase(0)
net_model = load_model(weight_file_path)

pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(net_model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

output nodes names are:  ['output_node0']


In [23]:
# Convert variables to constants and save
sess = K.get_session()
saver = tf.train.Saver()

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
#graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
save_path = saver.save(sess, "tf_models/hamlet.ckpt")
#print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))

saved the graph definition in ascii format at:  tf_models/hamlet.pb.ascii
INFO:tensorflow:Froze 5 variables.
Converted 5 variables to const ops.
